In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os
import bcolz
import numpy as np
import pickle
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import time
import math
import pickle
from tqdm import tqdm
from torch.autograd import Variable
# plotting
import matplotlib
matplotlib.use('Agg')
from vad import batchData
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

"""
Set seed #
"""
seed = 1337

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

def loadDataset(path = '../Datasets/Reviews/dataset_ready.pkl'):
    return pickle.load(open(path, 'rb'))

In [2]:
print("Loading parameters..", end=" ")
hiddenSize = 512
latentSize = 400
batchSize  = 32
iterations = 3
learningRate = 0.0001
bidirectionalEncoder = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Done.")

print("Loading dataset..", end=" ")
dataset = loadDataset()
# setup store parameters
id2word = dataset['id2word']
word2id = dataset['word2id']
weightMatrix = dataset['weights']
train = dataset['train']
validation = dataset['validation']
cutoff = dataset['cutoff']
paddingID = word2id['<pad>']
print("Done.")

print("Converting dataset weights into tensors..", end=" ")
# convert dataset into tensors
weightMatrix = torch.tensor(weightMatrix, dtype=torch.float)
print("Done.")

# batching data
print("Batching Data..",end=" ")

# shuffle data rows
random.shuffle(train)
random.shuffle(validation)

trainx = [x[0] for x in train]
trainy = [x[1] for x in train]
valx = [x[0] for x in validation]
valy = [x[1] for x in validation]

# shuffle data row

trainx = batchData(trainx, paddingID, device, batchSize, cutoff)
trainy = batchData(trainy, paddingID, device, batchSize, cutoff)
valx = batchData(valx, paddingID, device, batchSize, cutoff)
valy = batchData(valy, paddingID, device, batchSize, cutoff)

traindata = (trainx, trainy)
valdata= (valx, valy)
print("Done.")

# setup variables for model components initialisation
maxReviewLength = cutoff
vocabularySize = len(id2word)
embeddingDim = weightMatrix.shape[1]
embedding_shape = weightMatrix.shape

Loading parameters.. Done.
Loading dataset.. Done.
Converting dataset weights into tensors.. Done.
Batching Data.. Done.


In [3]:
from vad import Encoder, Decoder, Attn, Backwards, Prior, Inference, loss_function

print("Initialising model components..", end=" ")

modelEncoder = Encoder(weightMatrix, vocabularySize,
                       paddingID, hiddenSize, bidirectionalEncoder).to(device)
# modelAttention = Attention(maxLength=maxReviewLength).to(device)
modelAttention = Attn(hiddenSize, bidirectionalEncoder).to(device)
modelBackwards = Backwards(weightMatrix, vocabularySize,
                           paddingID, hiddenSize, bidirectionalEncoder).to(device)
modelInference = Inference(
    hiddenSize, latentSize, bidirectionalEncoder).to(device)
modelPrior = Prior(hiddenSize, latentSize, bidirectionalEncoder).to(device)
modelDecoder = Decoder(weightMatrix, vocabularySize,
                       paddingID, batchSize, maxReviewLength, hiddenSize, latentSize, bidirectionalEncoder).to(device)
criterion = nn.NLLLoss(ignore_index=paddingID)
print("Done.")

filepath = "attempt_1/"
# load models
print("Loading model weights..", end=" ")
modelEncoder.load_state_dict(torch.load(os.path.join(filepath,'encoder.pth')))
modelAttention.load_state_dict(torch.load(os.path.join(filepath,'attention.pth')))
modelBackwards.load_state_dict(torch.load(os.path.join(filepath,'backwards.pth')))
modelInference.load_state_dict(torch.load(os.path.join(filepath,'inference.pth')))
modelPrior.load_state_dict(torch.load(os.path.join(filepath,'prior.pth')))
modelDecoder.load_state_dict(torch.load(os.path.join(filepath,'decoder.pth')))
print("Done.")

Initialising model components.. Done.
Loading model weights.. Done.


In [4]:
def evaluate(x,
             xLength,
             y,
             yLength,
             encoder,
             attention,
             backwards,
             inference,
             prior,
             decoder,
             criterion
            ):
    
    loss = 0
    
    # initalise input and target lengths
    inputLength = x[0].size(0)
    targetLength = y[0].size(0)
    batchSize = x.shape[0]

    # set up encoder computation
    encoderHidden = encoder.initHidden(batchSize).to(device)
    backwardHidden = backwards.initHidden(batchSize).to(device)
    
    # set up encoder outputs
    encoderOutputs, encoderHidden = encoder(x, encoderHidden, xLength)

    # compute backwards outputs
    backwardOutput, backwardHidden = backwards(torch.flip(
        y, [0, 1]), yLength, backwardHidden)

    # set up the variables for decoder computation
    decoderInput = torch.tensor([[word2id["<sos>"]]] * batchSize, dtype=torch.long, device=device)
    
    decoderHidden = encoderHidden[-1]
    decoderOutput = None
    decoderOutputs = []
    
    # Run through the decoder one step at a time. This seems to be common practice across
    # all of the seq2seq based implementations I've come across on GitHub.
    for t in range(yLength[0]):
        # get the context vector c
        c = attention(encoderOutputs, decoderHidden)
        
        # compute the inference layer
        z_infer, infer_mu, infer_logvar = inference(decoderHidden, c, backwardOutput[:,t])
        
        # compute the prior layer
        z_prior, prior_mu, prior_logvar = prior(decoderHidden, c)
    
        # compute the output of each decoder state
        DecoderOut = decoder(decoderInput, c, z_prior, decoderHidden)
        
        # update variables
        decoderOutput, decoderHidden = DecoderOut
        decoderOutputs.append(decoderOutput)
        seqloss = loss_function(decoderOutput, y[:, t], infer_mu, infer_logvar, prior_mu, prior_logvar, criterion)
        loss += seqloss

        # feed this output to the next input
        decoderInput = y[:,t]
        decoderHidden = decoderHidden.squeeze(0)
        
    loss = loss.item()/targetLength
        
    return decoderOutputs, loss

In [31]:
def evaluateModel(batched_data,
                   encoder,
                   backward,
                   attention,
                   inference,
                   prior,
                   decoder,
                   criterion,
                   id2word,
                  ):
    
    encoder.eval()
    backward.eval()
    attention.eval()
    inference.eval()
    prior.eval()
    decoder.eval()
    numbatches = len(batched_data)
    for batch in tqdm(range(0,3)):
        # each batch is composed of the 
        # reviews, and a sentence length.
        x, xLength = batched_data[0][batch][0], batched_data[0][batch][1]
        y, yLength = batched_data[1][batch][0], batched_data[1][batch][1]
        
        outputs, losses = evaluate(x,
                                   xLength,
                                   y,
                                   yLength,
                                   encoder,
                                   attention,
                                   backward,
                                   inference,
                                   prior,
                                   decoder,
                                   criterion)
        
        print(y.shape)
        reference = convertRealID2Word(id2word,y)
        words = convertDecoderID2Word(id2word,outputs)

        for i in range(len(reference)):
            print("REFERENCE:", reference[i])
            print("MODEL:    ",words[i])
            print()
        
        print("LOSS:", losses)
#         break

def convertRealID2Word(id2word, y):
    entries = [[id2word[x.item()] for x in y[entry].cpu()] for entry in range(len(y))]
    for i in range(len(entries)):
        entries[i] = " ".join(entries[i][12:])
        entries[i] = entries[i].replace("<pad> ", "")
#         entries[i] = entries[i][43:]
    return entries
        
def convertDecoderID2Word(id2word, outputs):
    entries = []
    for batch_line in outputs:
        entry = [torch.argmax(batch_line[i]).cpu().item() for i in range(len(batch_line))]
        entries.append([id2word[i] for i in entry])
     
    words = []
    for i in range(len(outputs[0])):
#         line = "".join([entries[j][i] for j in range(0,12)])

        line = " ".join([entries[j][i] for j in range(12,len(entries))])
        line = line.replace(" ' ", "'")
        line = line.replace(" ) ", ")")
        line = line.replace(" . ", ".")
        line = line.replace(" , ", ", ")
        line = line.replace(" ( ", "(")
        words.append(line)
    return words




In [32]:
outputs = evaluateModel(valdata,
                   modelEncoder,
                   modelBackwards,
                   modelAttention,
                   modelInference,
                   modelPrior,
                   modelDecoder,
                   criterion,
                   id2word
                  )

  0%|          | 0/3 [00:00<?, ?it/s]

torch.Size([32, 60])


 33%|███▎      | 1/3 [00:01<00:02,  1.13s/it]

REFERENCE: i don ' t have a problem with american companies sourcing their products from asia ( i love asian electronics , like my samsung <unk> , nexus 10 , and tv ! ) , but they should at least get a little involved in the process , such
MODEL:     i'' t know to problem with the, ,, products, the, but'the)) but the <unk> <unk>, but, , <unk> the)).i i are be least of a lot more to the <unk>.but

REFERENCE: this keeps me safe from all the distractions ( some of whom i love very much ! ) from interrupting my rare & <unk> & <unk> time . all in all , it ' s a fun little gadget that will definitely be getting some use on my
MODEL:     i is the to for the the time <unk> <unk> of the )')))). the, <unk> of <unk> <unk> <unk> &.. the all, i's a great of to, is be be a.of.the

REFERENCE: it has had no problem wrecking every game i throw at it in <unk> on full settings ( bf3 , <unk> <unk> , crysis , etc ) . it ' s also an excellent card for mining <unk> <unk> , easily pulling 700 <unk> for <unk> 

 67%|██████▋   | 2/3 [00:02<00:01,  1.13s/it]

REFERENCE: i think this is because it doesn ' t have a place for me to give it a profile of what my up/down speed is so with qos enabled even with only one device on the network i can ' t seem to get above <unk> on my
MODEL:     if'this is a a'' t work a problem to the, use it to try, the i <unk> are is not much the.. i the of of.the <unk>.have't use to be the to.the

REFERENCE: but i am disappointed with this item and looking straight into replacing it after just 2 months of use . i was using windows notebooks for most of my computing life till 2 years back when i need a light weight notebook of 2 . 5 kg or
MODEL:     polarity_0.0 a'using with the product.the for to the.is the the months of use.<eos> have able the 7, a years the <unk> and and.. ago.i got to new of.. the.5..

REFERENCE: the asus router claimed directional capability , and i did get better results by pointing the antennas toward rooms where i encountered the weakest signal . when i say the linksys doubled the coverage i

100%|██████████| 3/3 [00:03<00:00,  1.13s/it]

REFERENCE: traditionally , we use the line of otterbox products when it comes to enclosures , but with my recent foray with <unk> power related products ( car charges , and external battery backups ) i decided this was an excellent opportunity to expand that product vendor line .
MODEL:     i % the'the <unk> to the, , i comes to be the and i the <unk> <unk>, the, ,, , <unk>), <unk> the))).'to one a issue choice to get the <unk>.. .

REFERENCE: ios , android , or a computer ( pc or mac ) this thing is so much better than that clunky roku that developers don ' t take the time to iron out app issues due to better technology coming along , such as the <unk> . <eos>
MODEL:     i is the, and <unk> <unk>, <unk>, <unk>), is is a small easier than the',, is to't have you <unk> to use it of ,.. the than.. the and as a other, <unk>

REFERENCE: first the shortcomings . . . the kit lens is pretty soft and slow and i find it pretty much not acceptable . . . and the way olympus has hid the super cont